In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import BinaryClassifierOutputTarget
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, deprocess_image, preprocess_image
import numpy as np
import cv2
from torch.utils.data import DataLoader
from pathlib import Path

from landnet.config import MODELS_DIR
from landnet.enums import Mode, GeomorphometricalVariable, Architecture
from landnet.modelling import torch_clear
from landnet.modelling.classification.lightning import LandslideImageClassifier
from landnet.modelling.classification.models import get_architecture
from landnet.modelling.classification.dataset import ConcatLandslideImageClassification, LandslideImageClassification, create_classification_dataloader
from landnet.features.grids import get_grid_for_variable
from landnet.features.tiles import TileConfig, TileSize
from landnet.modelling.dataset import (
    get_default_transform,
    get_default_augment_transform,
)
from landnet.typing import TuneSpace

/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [2]:
checkpoint = MODELS_DIR / 'convnext_100x100_10vars/convnext_100x100_10vars/TorchTrainer_cf20a57d_5_batch_size=4,learning_rate=0.0000,tile_config=ref_ph_c793cfd2_2025-06-28_00-30-26/checkpoint_000016/checkpoint.ckpt'
assert checkpoint.exists()

tile_config = TileConfig(TileSize(100, 100), overlap=0)
model_config: TuneSpace = {
    'batch_size': 1,
    'learning_rate': 0.000001,
    'tile_config': tile_config,
}

variables = [
    GeomorphometricalVariable.HILLSHADE,
    GeomorphometricalVariable.TOPOGRAPHIC_POSITION_INDEX,
    GeomorphometricalVariable.NEGATIVE_TOPOGRAPHIC_OPENNESS,
    GeomorphometricalVariable.DIGITAL_ELEVATION_MODEL,
    GeomorphometricalVariable.EASTNESS,
    GeomorphometricalVariable.SLOPE,
    GeomorphometricalVariable.REAL_SURFACE_AREA,
    GeomorphometricalVariable.FLOW_LINE_CURVATURE,
    GeomorphometricalVariable.TERRAIN_RUGGEDNESS_INDEX,
    GeomorphometricalVariable.LOCAL_CURVATURE,
]
classifier = LandslideImageClassifier.load_from_checkpoint(
    checkpoint,
    model=get_architecture(Architecture('convnext'))(len(variables), Mode.INFERENCE),
)

DEBUG: Adapting model input channels from 10 to 3


In [3]:
grids = [
    get_grid_for_variable(
        variable,
        tile_config=tile_config,
        mode=Mode.TEST,
    )
    for variable in variables
]

dataset = ConcatLandslideImageClassification(
    landslide_images=[
        LandslideImageClassification(
            grid,
            Mode.TEST,
            transform=get_default_transform(),
        )
        for grid in grids
    ],
    augment_transform=None
)

INFO: Took 2.207024 seconds to compute data indices for TileSize(width=100, height=100) at mode='test'. Length of classes: {1: 193, 0: 207}


/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


INFO: Took 2.118966 seconds to compute data indices for TileSize(width=100, height=100) at mode='test'. Length of classes: {1: 193, 0: 207}


/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


INFO: Took 2.252890 seconds to compute data indices for TileSize(width=100, height=100) at mode='test'. Length of classes: {1: 193, 0: 207}


...

INFO: Took 2.614729 seconds to compute data indices for TileSize(width=100, height=100) at mode='test'. Length of classes: {1: 193, 0: 207}


/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


INFO: Took 2.386716 seconds to compute data indices for TileSize(width=100, height=100) at mode='test'. Length of classes: {1: 193, 0: 207}


/home/alex/miniforge3/envs/landnet/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


INFO: Took 2.385035 seconds to compute data indices for TileSize(width=100, height=100) at mode='test'. Length of classes: {1: 193, 0: 207}


In [4]:
dataloader = DataLoader(
    dataset,
    batch_size=model_config['batch_size'],
    num_workers=4,
    shuffle=False,
    prefetch_factor=4,  # Load 4 batches ahead
    persistent_workers=True,  # Keeps workers alive
    #class_balance=DEFAULT_CLASS_BALANCE,
    pin_memory=True,
)

tensors = list(dataloader)

In [ ]:
torch_clear()

method = ScoreCAM

target_layers = classifier.model[-1].features[-1]
# Note: input_tensor can be a batch tensor with several images!

# We have to specify the target we want to generate the CAM for.
tensors_list = [tensor for tensor in tensors[0][0]]

# Construct the CAM object once, and then re-use it on many images.
for i, input_tensor in enumerate(tensors):
    print(i)
    targets = [BinaryClassifierOutputTarget(1) for target in input_tensor[1]]
    visualizations = []
    with method(model=classifier, target_layers=target_layers) as cam:
        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        grayscale_cams = cam(input_tensor=input_tensor[0], targets=targets)
    
        # In this example grayscale_cam has only one image in the batch:
        for img_index in range(input_tensor[0].shape[0]):
            # hillshade = np.stack((input_tensor[0][img_index, 0].numpy(),)*3, axis=-1)
            # cam_image = show_cam_on_image(hillshade, grayscale_cam, colormap=cv2.COLORMAP_RAINBOW)
            # cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)
    
            grayscale_cam = grayscale_cams[img_index, :]
            visualizations.append(grayscale_cam)
        # You can also get the model outputs without having to redo inference

    for j, vis in enumerate(visualizations):
        index = i * model_config['batch_size'] + j
        vis_resized = np.expand_dims(cv2.resize(vis, (100, 100)), 0)
        out_file = Path(f'./{method.__name__}/{input_tensor[1][j]}_{index}.png')
        out_file.parent.mkdir(exist_ok=True)
        grids[0].write_tile(index, vis_resized, out_dir=out_file.parent, prefix=out_file.with_suffix('.tif').name)
        #cv2.imwrite(out_file, vis)

0





00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:18<00:00,  3.50it/s]

1





00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:18<00:00,  3.55it/s]

2


...




00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:18<00:00,  3.54it/s]

24





00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:18<00:00,  3.44it/s]

25



 44%|██████████████████████████████████████████████████████████████████▉                                                                                      | 28/64 [00:07<00:10,  3.56it/s]